![alt text](img/LM.png)
# Kurs: Deep Learning, Text Mining i XAI w Pythonie

## Autor: Piotr Ćwiakowski

### Lekcja 1. Wyjaśnialne uczenie maszynowe - ćwiczenia

## Zadanie 1.

Wczytaj zbiór `titanic2.csv`. Opis zbioru poniżej:

Nazwa zmiennej | Opis 
:------------- | :--------------------------------------
Pclass         | Passenger class (1 = 1st; 2 = 2nd; 3 = 3rd)
Survived       | Survival (0 = No; 1 = Yes)
name           | Passenger name
sex            | Passenger gender
age            | Passenger age
sibsp          | Number of Siblings/Spouses Aboard of a Passenger
parch          | Number of Parents/Children Aboard of a Passenger
ticket         | Ticket Number
fare           | Passenger Fare (British pound)
cabin          | Cabin Number
embarked       | Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
boat           | Lifeboat
body           | Body Identification Number
home.dest      | Home/Destination

Wykonaj następujące polecenia:

* przygotuj zbiór do analizy (sprawdź czy są braki danych, usuń nieprzydatne zmienne oraz zmienne stanowiące przeciek informacji, zamień zmienne nominalne na binarne etc.) oraz podziel zbiór na treningowy i testowy. Przygotuj obiekty `features` i `target`.

In [1]:
# Podstawowe pakiety
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ustawienia graficzne
sns.set(style='whitegrid')
sns.set_palette("bright")
%matplotlib inline

# Machine Learning
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score

# Modele ML
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor
from helpers import run_xgb

# Selekcja zmiennych
from boruta import BorutaPy
from sklearn.feature_selection import RFE

# XAI
from pdpbox import pdp, info_plots
from lime import lime_tabular
import dalex as dx

In [2]:
# Wczytanie danych
titanic = pd.read_csv('data/titanic2.csv', sep=';', decimal=',')

['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_male', 'embarked_Q', 'embarked_S']


,pclass,survived,age,sibsp,parch,fare,sex_male,embarked_Q,embarked_S
0,1,1,29.0000,0,0,211.3375,0,0,1
1,1,1,0.9167,1,2,151.5500,1,0,1
2,1,0,2.0000,1,2,151.5500,0,0,1
3,1,0,30.0000,1,2,151.5500,1,0,1
4,1,0,25.0000,1,2,151.5500,0,0,1


(836, 8) (209, 8)


* wykonaj model przewidujący przeżycie pasażerów na statku Titanic wykorzystując logit, lasy losowe, xgboost oraz dowolne inne znane Ci modele (np. SVM lub KNN). Policz metryki dopasowania dla tych modeli (Accuracy, ROCAUC, Precyzję, Czułość). Dodatkowo możesz spróbować narysować krzywą ROC (można to dość prosto zrobić wykorzystując pakiet Dalex - wyszukaj w Internecie odpowiednie polecenia). Dla modeli opartych o drzewa decyzyjne (z ang. *tree based*) policz różne rankingi ważności zmiennych. Czy wiesz jak stworzyć podobny ranking dla pozostałych metod?

* Dla kilku najważniejszych (i najlepiej ciągłych) zmiennych policz oczekiwane odpowiedzi dla różnych modeli na podstawie historycznych danych. Następnie dla najlepszego modelu policz kontrfaktyczne wykresy PDP, ALE, ICE (dla tych samych zmiennych).  Zinterpretuj wyniki. Zadanie wykonaj dla zbioru testowego, jeśli okaże się za mały, wygeneruj prognozy dla całego zbioru. Czy między zmiennymi zachodzą ciekawe interakcje?

* Wybierz z danych (ze zbioru testowego):
    - 2 pasażerów, którzy przeżyli i model prognozuje ten fakt poprawnie,
    - 2 pasażerów, którzy przeżyli i model prognozuje ten fakt błędnie,
    - 2 pasażerów, którzy nie przeżyli i model prognozuje ten fakt poprawnie,
    - 2 pasażerów, którzy nie przeżyli i model prognozuje ten fakt błędnie.
    
Nastepnie na podstawie wykresu Breakdown, wartości Shapleya, lokalnych surogatów (LIME) oraz wykresów Ceteris Pariubs (dla najważniejszych zmiennych) spróbuj zastanowić się, dlaczego model popełnia w niektórych przypadkach błędy.    

* Spróbuj poprawić model regresji logistycznej za pomocą zmiennych przekształconych modelem lasów losowych.

# Zadanie 2.

Wczytaj zbiór apartments. Wykonaj następujące polecenia:

In [3]:
import dalex as dx
import pandas as pd
import numpy as np

data = dx.datasets.load_apartments()
data.head()

,m2_price,construction_year,surface,floor,no_rooms,district
1,5897,1953,25,3,1,Srodmiescie
2,1818,1992,143,9,5,Bielany
3,3643,1937,56,1,2,Praga
4,3517,1995,93,7,3,Ochota
5,3013,1992,144,6,5,Mokotow


* przygotuj zbiór do analizy (sprawdź czy są braki danych, usuń nieprzydatne zmienne, zamień zmienne nominalne na binarne etc.) oraz podziel zbiór na treningowy i testowy. Przygotuj obiekt `features` i `target`.

* wykonaj model prognozujący ceny nieruchomości (`m2_price`) wykorzystując regresję liniową, lasy losowe, xgboost oraz dowolne inne znane Ci modele (np. SVM lub KNN). Policz metryki dopasowania dla tych modeli (RMSE, MAE, MAPE, RMSLE). Dla modeli opartych o drzewa decyzyjne (z ang. *tree based*) policz różne rankingi ważności zmiennych. Czy wiesz jak stworzyć podobny ranking dla pozostałych metod?

* Wykorzystując pakiet Dalex, dokonaj analizy rozkładu reszt w różnych modelach. Zinterpretuj wyniki

* Wykonaj wykresy PDP, ALE, ICE i porównaj sposób działania zmiennych w różnych modelach. Zinterpretuj wyniki. Czy między zmiennymi zachodzą ciekawe interakcje?

Sprawdź mechanizm generowania prognoz dla kilku obserwacji z różnych obszarów rozkładu zmiennej objaśnianej (np. lewy ogon, środek, prawy ogon) za pomocą wykresu Breakdown, wartości Shapleya, lokalnych surogatów (LIME) oraz wykresów `What-if`. Czy w różnych półkach cenowych zmienne oddziałują odmiennie?

* spróbuj poprawic model regresji liniowej wykorzystując do tego zmienne przekształcone za pomocą modelu lasów losowych.